Load data

In [ ]:
import json


file_path = '12-11-game-states.json'

game_states = []
with open(file_path, 'r') as file:
    for line in file:
        game_states.append(json.loads(line.strip()))

print(f"Loaded {len(game_states)} game states.")

Loaded 10596 game states.
